### Problem Set 1
5. Write down three equations. One with just a "treatment period" dummy, one with
just a "treatment city" dummy and then one that adds both of them with their
interaction. This last regression gives you a dierence-in-dierence estimate of the
eect of the event on prices. Explain which coecient captures this treatment eect
and why you need a second city for this. (3 points)

6. Estimate all three regressions. Make a standard regression table with 4 columns (3 for
your answer here and one more below). Make sure you check how these regressions
look like usually. Always report all coecients. Then carefully interpret them for each
regression and the changes you see. (4 points)

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import statsmodels.api as sm
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

In [2]:
# # load the CSV final file into a DataFrame
# file_path = 'booking_final.csv'
# df = pd.read_csv(file_path)

# df.head()


In [3]:
file_regression = '20240203_booking_final_regression.csv'
df_regression = pd.read_csv(file_regression)

df_regression.head()

,name,rating,num_reviews,neighborhood,dist_from_center,price,other,is_Barna,is_Merce_time,short_description,descriptions,Free_Cancellation,Breakfast_Included,No_Prepayment,Facilities
0,Room Mate Gerard,8.8,2783.0,Eixample,700,1730,"Subway Access, Beach Nearby",1,1,Junior Suite\nPrivate suite • 1 bedroom • 1 li...,You're eligible for a Genius discount at Room ...,0,1,0,0.216086
1,Sonder Los Arcos,8.4,149.0,Ciutat Vella,1000,1530,"Subway Access, Beach Nearby",1,1,Queen Room with Two Queen Beds\n2 queen beds\n...,Ideally located in the Ciutat Vella district o...,0,0,0,-2.184302
2,Occidental Barcelona 1929,8.9,4066.0,Montjuïc,2300,909,Subway Access,1,1,Superior Double Room\nBeds: 1 double or 2 twin...,You're eligible for a Genius discount at Occid...,0,0,0,-0.061278
3,Hotel Alimara,8.3,3892.0,Guinardó,5500,749,Subway Access,1,1,Double Room\n1 queen bed\nFree cancellation,You're eligible for a Genius discount at Hotel...,1,0,0,0.749370
4,Weflating City Center,8.8,1573.0,Eixample,500,978,"Subway Access, Beach Nearby",1,1,Economy Double Room\n2 bunk beds\nFree cancell...,You're eligible for a Genius discount at Wefla...,1,0,0,-0.717773


In [4]:
#features and target y value
X = df_regression[['is_Merce_time', 'is_Barna', 'Free_Cancellation', 'Breakfast_Included', 'No_Prepayment', 'Facilities']].copy()
y = df_regression['price']

In [5]:
# regression for the treatment Period dummy
model1 = LinearRegression().fit(X[['is_Merce_time']], y)

In [6]:
# regression for the treatment city dummy
model2 = LinearRegression().fit(X[['is_Barna']], y)

In [7]:
# regression combined with interaction
X['EventPeriod_Barcelona'] = X['is_Merce_time'] * X['is_Barna']

model3 = LinearRegression().fit(X[['is_Merce_time', 'is_Barna', 'EventPeriod_Barcelona']], y)

In [8]:
print(X.head())

   is_Merce_time  is_Barna  Free_Cancellation  Breakfast_Included  \
0              1         1                  0                   1   
1              1         1                  0                   0   
2              1         1                  0                   0   
3              1         1                  1                   0   
4              1         1                  1                   0   

   No_Prepayment  Facilities  EventPeriod_Barcelona  
0              0    0.216086                      1  
1              0   -2.184302                      1  
2              0   -0.061278                      1  
3              0    0.749370                      1  
4              0   -0.717773                      1  


In [9]:
# regression combined with interaction and control variables
model4 = LinearRegression().fit(X, y)

In [10]:
print("Model 1 Coefficients:", model1.coef_)
print("Model 1 Intercept:", model1.intercept_)

print("\nModel 2 Coefficients:", model2.coef_)
print("Model 2 Intercept:", model2.intercept_)

print("\nModel 3 Coefficients:", model3.coef_)
print("Model 3 Intercept:", model3.intercept_)

print("\nModel 4 Coefficients:", model4.coef_)
print("Model 4 Intercept:", model4.intercept_)


Model 1 Coefficients: [-65.38299621]
Model 1 Intercept: 1557.678571428572

Model 2 Coefficients: [637.37081341]
Model 2 Intercept: 1067.3734567901233

Model 3 Coefficients: [-34.308387   651.40627227 -30.22893809]
Model 3 Intercept: 1084.6335403726716

Model 4 Coefficients: [ -27.40743294  876.85171131 -137.77724561  562.41614083   95.22150961
   43.16167892  -36.27322717]
Model 4 Intercept: 846.44222176456


In [11]:
# here we use stargazer's OLS functionality so that we can produce our desired standard regression table

mod1 = sm.OLS(endog=y, exog=sm.add_constant(X[['is_Merce_time']])).fit()
#model 1 OLS with just the Merce_time var

mod2 = sm.OLS(endog=y, exog=sm.add_constant(X[['is_Barna']])).fit()
#model 2 OLS with just the is_Barna var

mod3 = sm.OLS(endog=y, exog=sm.add_constant(X[['is_Merce_time', 'is_Barna', 'EventPeriod_Barcelona']])).fit()
#model 3 OLS with first two variables and the interaction term

mod4 = sm.OLS(endog=y, exog=sm.add_constant(X)).fit()
# #model 4 OLS with first two variables, interaction term and the control variables


stargazer = Stargazer([mod1, mod2, mod3, mod4])

In [12]:
stargazer_tab = Stargazer([mod1, mod2, mod3, mod4])
stargazer_tab

In [13]:
stargazer.custom_columns(['Model 1', 'Model 2', 'Model 3', 'Model 4'], [1, 1, 1, 1])

# render the table in latex
latex_table = stargazer.render_latex()

In [14]:
latex_table

'\\begin{table}[!htbp] \\centering\n\\begin{tabular}{@{\\extracolsep{5pt}}lcccc}\n\\\\[-1.8ex]\\hline\n\\hline \\\\[-1.8ex]\n& \\multicolumn{4}{c}{\\textit{Dependent variable: price}} \\\n\\cr \\cline{2-5}\n\\\\[-1.8ex] & \\multicolumn{1}{c}{Model 1} & \\multicolumn{1}{c}{Model 2} & \\multicolumn{1}{c}{Model 3} & \\multicolumn{1}{c}{Model 4}  \\\\\n\\\\[-1.8ex] & (1) & (2) & (3) & (4) \\\\\n\\hline \\\\[-1.8ex]\n Breakfast_Included & & & & 562.416$^{***}$ \\\\\n& & & & (168.104) \\\\\n EventPeriod_Barcelona & & & -30.229$^{}$ & -36.273$^{}$ \\\\\n& & & (245.661) & (245.825) \\\\\n Facilities & & & & 43.162$^{}$ \\\\\n& & & & (56.667) \\\\\n Free_Cancellation & & & & -137.777$^{}$ \\\\\n& & & & (155.723) \\\\\n No_Prepayment & & & & 95.222$^{}$ \\\\\n& & & & (192.055) \\\\\n const & 1557.679$^{***}$ & 1067.373$^{***}$ & 1084.634$^{***}$ & 846.442$^{***}$ \\\\\n& (78.132) & (104.059) & (147.729) & (178.561) \\\\\n is_Barna & & 637.371$^{***}$ & 651.406$^{***}$ & 876.852$^{***}$ \\\\\n& &